<a href="https://colab.research.google.com/github/leegonzales/LangChainExamples/blob/main/Public_Copy_Langchain_transcription_with_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## V1 Query the YouTube video transcripts, returning timestamps as sources to legitimize the answers by [@m_morzywolek](https://twitter.com/m_morzywolek)

## V2 with custom prompts and summaries included by [@leegonzales](https://twitter.com/leegonzales)


In [ ]:
# First set runtime to GPU

In [ ]:
pip install pytube # For audio downloading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 1.1 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/openai/whisper.git -q # Whisper from OpenAI transcription model

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [ ]:
import whisper 
import pytube 

In [ ]:
url = "https://www.youtube.com/watch?v=8jEmIDwqnL4"
video = pytube.YouTube(url)

In [ ]:
audio = video.streams.get_audio_only()
audio.download(filename='tmp.mp3') # Downlods only audio from youtube video

'/content/tmp.mp3'

In [ ]:
model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 191MiB/s]


In [ ]:
transcription = model.transcribe('/content/tmp.mp3')

In [ ]:
res = transcription['segments']

In [ ]:
from datetime import datetime

def store_segments(segments):
  texts = []
  start_times = []

  for segment in segments:
    text = segment['text']
    start = segment['start']

    # Convert the starting time to a datetime object
    start_datetime = datetime.fromtimestamp(start)

    # Format the starting time as a string in the format "00:00:00"
    formatted_start_time = start_datetime.strftime('%H:%M:%S')

    texts.append("".join(text))
    start_times.append(formatted_start_time)

  return texts, start_times

In [ ]:
store_segments(res)

([" Hey everybody, Peter Zayn here. We've had a lot of questions come in about AI and what that means for the workforce moving forward.",
  ' What sort of activities should we expect to be replaced? What does this mean for economics and labor and politics?',
  ' And are there any obvious winners either in terms of geography or sectors?',
  ' What popular one that comes in is whether or not this is going to hit red states or blue states more, for example.',
  " The cop-out answer is we don't really know yet because we're dealing with technologies that have yet to be invented.",
  ' But there are a few general guidelines we have.',
  " First of all, it's not so much that jobs get created or destroyed. It's that they change.",
  " And it's pretty common when you're dealing with an environment that has evolved because of technology, you know, jobs evolve too.",
  " We've been talking about technology overwhelming the workforce really since the onset of the Industrial Revolution.",
  " And 

In [ ]:
texts, start_times = store_segments(res)

In [ ]:
pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.0
    Uninstalling SQLAlchemy-2.0.0:
      Successfully uninstalled SQLAlchemy-2.0.0


In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.4-py3-none-any.whl size=67744 sha256=c756543a37099ee7b696d98330f48db3f4ccab3e300ccfd10ee5e732a83c3a2f
  Stored in directory: /root/.cache/pip/wheels/2b/d8/4e/268f029bd3277c1dd9e8781a0e0296e0a63822665bfa2429fc
Successfully built openai


In [ ]:
pip install --upgrade faiss-gpu==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 MB 12.7 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain import OpenAI
import openai
import faiss

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass('Enter your OpenAI key: ')

Enter your OpenAI key: ··········


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = []
metadatas = []
for i, d in enumerate(texts):
    splits = text_splitter.split_text(d)
    docs.extend(splits)
    metadatas.extend([{"source": start_times[i]}] * len(splits))
embeddings = OpenAIEmbeddings()

In [ ]:
store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)
faiss.write_index(store.index, "docs.index")

In [ ]:
chain = VectorDBQAWithSourcesChain.from_llm(llm=OpenAI(temperature=0), vectorstore=store)

In [ ]:
result = chain({"question": "For each major point Peter raises, where is he wrong?"})

Token indices sequence length is longer than the specified maximum sequence length for this model (1596 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
print(f"Answer: {result['answer']}  Sources: {result['sources']}")

Answer:  I don't know.
  Sources: 00:07:30, 00:02:30, 00:06:00, 00:09:01


In [ ]:
from langchain.docstore.document import Document
summaryDoc = [Document(page_content=t) for t in texts[:3]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
prompt_template = """Write a complete and thorough synopsis of the following, include a bulleted list of his points:


{text}

Output using the following format
SYNOPSIS:

KEY POINTS:"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm=OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain.run(summaryDoc)

'\n- Peter Zayn is discussing the implications of Artificial Intelligence (AI) on the workforce.\n- He is exploring what activities will be replaced by AI, and what this means for economics, labor, and politics.\n- He is also considering which geographical areas and sectors will benefit from the introduction of AI.'

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm=OpenAI(temperature=0), chain_type="stuff")
chain.run(summaryDoc)

' Peter Zayn is discussing the implications of AI on the workforce, economics, labor, and politics. He is asking what activities will be replaced, and who will benefit from the changes.'